## LLM Math

In [1]:
import os
os.environ["OPENAI_API_KEY"] = 'openai api key'

In [2]:
from langchain import OpenAI, LLMMathChain

llm = OpenAI(temperature=0)
llm_math = LLMMathChain.from_llm(llm, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")

Could not import azure.core python package.




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```text
13 ** .3432
```
...numexpr.evaluate("13 ** .3432")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

## LLM Bash

In [15]:
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

text = "Please write a bash script that prints 'Hello World' to the console."

bash_chain = LLMBashChain.from_llm(llm, verbose=True)

bash_chain.run(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```
Code: ['echo "Hello World"']
Answer: "Hello World"

> Finished chain.


'"Hello World"\r\n'

### Customize Prompt

In [16]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm_bash.prompt import BashOutputParser

_PROMPT_TEMPLATE = """If someone asks you to perform a task, your job is to come up with a series of bash commands that will perform the task. There is no need to put "#!/bin/bash" in your answer. Make sure to reason step by step, using this format:
Question: "copy the files in the directory named 'target' into a new directory at the same level as target called 'myNewDirectory'"
I need to take the following actions:
- List all files in the directory
- Create a new directory
- Copy the files from the first directory into the second directory
```bash
ls
mkdir myNewDirectory
cp -r target/* myNewDirectory
```

Do not use 'echo' when writing the script.

That is the format. Begin!
Question: {question}"""

PROMPT = PromptTemplate(input_variables=["question"], template=_PROMPT_TEMPLATE, output_parser=BashOutputParser())

In [17]:
bash_chain = LLMBashChain.from_llm(llm, prompt=PROMPT, verbose=True)

text = "Please write a bash script that prints 'Hello World' to the console."

bash_chain.run(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
printf "Hello World\n"
```
Code: ['printf "Hello World\\n"']
Answer: Command 'printf "Hello World\n"' returned non-zero exit status 1.
> Finished chain.


'Command \'printf "Hello World\\n"\' returned non-zero exit status 1.'

### Persistent Terminal

In [18]:
from langchain.utilities.bash import BashProcess


persistent_process = BashProcess(persistent=True)
bash_chain = LLMBashChain.from_llm(llm, bash_process=persistent_process, verbose=True)

text = "List the current directory then move up a level."

bash_chain.run(text)

ValueError: Persistent bash processes are not yet supported on Windows.

In [ ]:
# Run the same command again and see that the state is maintained between calls
bash_chain.run(text)

## Router Chains

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

Could not import azure.core python package.


In [3]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    }
]

In [5]:
llm = OpenAI()

In [6]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

### LLMRouterChain

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [11]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the thermal electromagnetic radiation emitted by a black body due to its temperature. A black body is an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. It is a perfect absorber, but it is also a perfect emitter, meaning that it emits a characteristic spectrum of radiation that depends only on its temperature. This spectrum is known as black body radiation and is the source of the ultraviolet catastrophe.


In [12]:
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?'}
> Finished chain.


The answer is 43. One plus 43 (44) is divisible by 3.


In [10]:
print(chain.run("What is the name of the type of cloud that rins"))



> Entering new MultiPromptChain chain...
None: {'input': 'What is the name of the type of cloud that rins'}
> Finished chain.
 The type of cloud that rains is called a cumulonimbus cloud. It's a type of thunderstorm cloud that is tall, dense, and vertically developed. It often produces heavy precipitation and strong winds.


### EmbeddingRouterChain

In [9]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma

In [10]:
names_and_descriptions = [
    ("physics", ["for questions about physics"]),
    ("math", ["for questions about math"]),
]

In [11]:
os.environ["COHERE_API_KEY"] ="cohere api key"

In [16]:
pip install cohere

  Using cached importlib_metadata-6.6.0-py3-none-any.whl (22 kB)
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
Note: you may need to restart the kernel to use updated packages.


In [12]:
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]
)

In [13]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [19]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



> Finished chain.


Black body radiation is the type of electromagnetic radiation that is emitted from a black body in thermodynamic equilibrium. A black body is an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. The emitted energy is a function of the temperature of the body, and is known as the Stefan-Boltzmann law.


In [14]:
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
?

The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To find this number, you can start by listing out the prime numbers greater than 40: 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97. Then, you can add 1 to each number to check if it is divisible by 3. The first number that is divisible by 3 is 43 + 1 = 44, so 43 is the answer.
